-Compute T(E,z_in, z_dep) by doing a delta function injection

-Why don't we ever keep track of how much DM is being depleted through annihilation/decay?

TODO:
- make contour plots
    - Pick a mass and make the plot corresponding to a vertical line going through the contour plot
    - Make a course contour plot
    - Once everything is debugged, make a very fine contour plot
- Clean up this Example12
    - migrate the main evolve function into a main.py that is conspicuous for the casual user
    - Change the inputs so that xe_init and T_init are computed for the user.
    - Make sure that the transfer functions are never seen (take them out of the input)
    - plots comparing back-reaction and no back-reaction
    - allow electron annihilation

This Notebook teaches the user to use the `main.evolve()` function.  Given a model of decaying or annihilating DM, `main.evolve()` will compute the free electron fraction as a function of redshift, $x_e(z) = n_e(z)/n_{HI}(z)$, and the baryon temperature as a function of redshift, $T_m(z)$.

# Full Transfer Functions

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [2]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from astropy.io import fits

import numpy as np
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

from darkhistory.low_energy.lowE_deposition import compute_fs
import main

# np.set_printoptions(threshold=np.nan)

In the standard temperature/ionization history calculation,
one solves the TLA assuming no DM energy injection. 

$$ \dot{T}_m = -2 H T_m + \Gamma_C \left(T_{CMB} - T_m \right), $$
$$ \dot{x}_e = - \mathcal{C} \left[ n_H x_e^2 \alpha_B - 4 (1 - x_e) \beta_B e^{-E_{21}/T_{CMB}} \right]. $$

The solution to these coupled ODEs provides $x_e$ and $T_m$ as functions of z, which are loaded below.

In [3]:
soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
xe_std  = interp1d(soln[0,:], soln[2,:])
T_m_std = interp1d(soln[0,:], soln[1,:])

To include the effect of DM energy injection, one adds to the TLA the following terms

$$ \dot{T}^\chi_m = \frac{2 f_\text{heat}(z)}{3 (1 + f_\text{He} + x_e(z)) n_H(z)} \left(\frac{dE}{dV \, dt} \right)^\text{inj}, $$
$$ \dot{x}^\chi_e = \left[\frac{f_\text{ion}(z)}{\mathcal{R} n_H(z)} + \frac{(1-\mathcal{C}) \; f_\text{exc}(z)}{0.75 \mathcal{R} n_H(z)} \right] \left(\frac{dE}{dV \, dt} \right)^\text{inj}. $$

Apparently one must know $f_{\{\text{heat, ion, exc}\}}(z)$ for all z to solve the TLA, which requires a knowledge of $x_e(z)$ and the histories of the DM decay/annihilation products.  Below we will either generate the $f_c$'s by using the standard history values ("without back-reaction"), or by including the effects of DM energy injection ("with back-reaction").

## Photon Transfer Functions

Transfer functions encode how particles (photons and electrons in our case) transfer energy as they evolve along with the universe.  For example, a high energy photon will lose energy through redshifting and scattering off of free electrons (what else am I missing???).  

Let $E_{in}^i$ be the energy of a single photon injected at redshift $z_{in}$. The photon produces $N^j$ photons at energies $E_{out}^j$ because of energy transfer processes that took place over $d(\ln(1+z))$ worth of cosmological time.  If we define $T^{ij} \equiv T(E_{in}^i, E_{out}^j)$, we have

$$ N_\gamma\left(E^j, (1+z_{in})e^{- d(\ln(1+z))} \right) = T^{ij} N_\gamma\left(E^i, 1+z_{in} \right), $$

where $N_\gamma(E, 1+z)$ represents the number of photons of energy E at redshift 1+z.  The transfer function allows us to evolve photon spectra $d(\ln(1+z))$ amount of time at a time.  We will set $d(\ln(1+z))$ = $0.002$ in our transfer functions, which has been shown to be small enough to converge.

We now load the transfer functions.

In [34]:
highengphot_tf_interp, lowengphot_tf_interp, lowengelec_tf_interp = main.load_trans_funcs()

Loading transfer functions...
Loaded high energy photons...
Low energy photons...
Low energy electrons...
CMB losses.

Padding tflists with zeros...
high energy photons...
low energy photons...
low energy electrons.

Generating TransferFuncInterp objects for each tflist...
Done.



The above transfer functions end in `_interp` because only a finite number of them have been generated ($11 \times 494$ to be exact.  11 choices of ionization level, $x_e$, and 494 choices of input energy, $E_{in}$), which means we have to perform a 2D interpolation over different $x_e$ and $E_{in}$.

We now turn to the main function in DarkHistory: main.evolve().  It

In [ ]:
%autoreload
end_rs   = 10
mDM = 1e8
sigmav = 1e-26
tau = 1.5e25


phot_spec_init = spectools.rebin_N_arr(np.array([1]), 
                                      np.array([mDM]), 
                                      highengphot_tf_interp.eng)
phot_spec_init.rs = 3000
phot_spec_init.switch_spec_type()

def rate_func_swave_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)/(2*mDM)

def rate_func_swave_eng(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)

def rate_func_decay_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('decay', rs, tau=tau)/mDM

def rate_func_decay_eng(rs):
    # This is dN/dV dt
    return phys.inj_rate('decay', rs, tau=tau)


def rate_func_swave_N_2(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)/(2*mDM)

def rate_func_swave_eng_2(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)

#!!! What about Reionization
a = main.evolve(
    phot_spec_init*0, phot_spec_init,
    rate_func_decay_N, rate_func_decay_eng, end_rs,
    highengphot_tf_interp, lowengphot_tf_interp, lowengelec_tf_interp,
    xe_init=xe_std(phot_spec_init.rs), Tm_init=phys.TCMB(phot_spec_init.rs), 
    coarsen_factor=1, std_soln=True
)

b = main.evolve(
    phot_spec_init*0, phot_spec_init,
    rate_func_decay_N, rate_func_decay_eng, end_rs,
    highengphot_tf_interp, lowengphot_tf_interp, lowengelec_tf_interp,
    xe_init=xe_std(phot_spec_init.rs), Tm_init=phys.TCMB(phot_spec_init.rs), 
    coarsen_factor=1, std_soln=False
)

start to evolve
rs, fs:  2994.005996001995   [5.010253871064946e-06, 3.139708532323484e-05, 0.0002113287238244386, 2.6324947675953795e-18, 0.002632474604939013]
rs, fs:  2988.023968031968   [2.0911031212181267e-05, 0.00011400683398819264, 0.0006591080663632203, 8.314652025152229e-18, 0.008314585865768418]
rs, fs:  2982.053892161797   [4.3113566113991716e-05, 0.0002177254112830476, 0.0011326374444918985, 1.4426114020448714e-17, 0.014425994774152788]
rs, fs:  2976.095744511174   [6.995630529748116e-05, 0.00033668626888452445, 0.0016229273624116102, 2.0844959714827643e-17, 0.020844780783461768]


../darkhistory/spec/spectrum.py:748: RuntimeWarning: The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.
  warnings.warn("The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.", RuntimeWarning)
../darkhistory/spec/spectrum.py:662: RuntimeWarning: divide by zero encountered in log
  np.log(bound_arr),


rs, fs:  2970.149501247497   [0.00010017927491325333, 0.00046641655079821947, 0.002123144617948969, 2.749636667420569e-17, 0.027496121629245987]
rs, fs:  2964.215138585785   [0.00013280651417066323, 0.0006034624394151668, 0.002628045542201952, 3.4330210546906635e-17, 0.03432989295674459]
rs, fs:  2958.29263278858   [0.0001670714820213938, 0.0007451358774270879, 0.0031335852289433603, 4.1309764944379617e-17, 0.041309368279188616]
rs, fs:  2952.38196016585   [0.00020236693922506734, 0.0008893355428200464, 0.0036366375365373886, 4.840595893794108e-17, 0.048405476514367676]
rs, fs:  2946.4830970748976   [0.00023820952454311085, 0.0010344157750051082, 0.004134788447687652, 5.559442498085523e-17, 0.05559384992756715]
rs, fs:  2940.5960199202623   [0.0002742136734504978, 0.0011790880684000203, 0.004626180238446625, 6.285395274872058e-17, 0.06285327798248963]
rs, fs:  2934.720705153627   [0.0003100718196207456, 0.0013223456518814995, 0.005109392145802281, 7.016565458826084e-17, 0.0701648727953

rs, fs:  2650.1395226482796   [0.0008915313034547977, 0.003962319187613061, 0.017052296560849514, 3.5008834048271044e-16, 0.3500601836624274]
rs, fs:  2644.8445403502756   [0.0008833214903975374, 0.003945625650256426, 0.017109392894418003, 3.530960684369057e-16, 0.35306648757671005]
rs, fs:  2639.5601374339594   [0.0008746507665007099, 0.003927380675772396, 0.01716238828140254, 3.5602347112336877e-16, 0.3559924007096291]
rs, fs:  2634.2862927617125   [0.0008655321468763349, 0.003907629698872543, 0.017211394875537342, 3.588720727378257e-16, 0.3588394445114208]
rs, fs:  2629.022985238149   [0.0008559783246636565, 0.0038864169724505395, 0.017256522000537074, 3.616433920664291e-16, 0.3616091346226128]
rs, fs:  2623.7701938100317   [0.0008460016764499527, 0.003863785592855527, 0.017297876211023685, 3.643389408848387e-16, 0.3643029789635496]
rs, fs:  2618.5278974661883   [0.000841704759924475, 0.003857061834326356, 0.017359816206822185, 3.6704313621916754e-16, 0.3670053829200613]
rs, fs:  26

rs, fs:  2355.168532655543   [0.0006674403863881047, 0.0035497383273746814, 0.01908461838447819, 4.389121910715978e-16, 0.43845604316651887]
rs, fs:  2350.4629027886426   [0.0006618174048195893, 0.0035350218590305127, 0.019083653588867388, 4.393593681752254e-16, 0.43888118734755244]
rs, fs:  2345.7666747764874   [0.0006561443753261519, 0.0035201159820056134, 0.019081946974230487, 4.3978621629173206e-16, 0.4392849189999112]
rs, fs:  2341.0798298341588   [0.0006504226996310435, 0.003505026181320485, 0.019079517580068518, 4.4019328499621984e-16, 0.439667728872341]
rs, fs:  2336.402349214271   [0.0006446537441381813, 0.0034897578090448654, 0.01907638398377767, 4.4058112065912267e-16, 0.44003009574092067]
rs, fs:  2331.734214206895   [0.0006388388409932821, 0.003474316088275025, 0.01907256431297676, 4.4095023811039198e-16, 0.4403724866560712]
rs, fs:  2327.075406139485   [0.0006329792891226396, 0.003458706117021607, 0.019068076257552104, 4.413011611483562e-16, 0.4406953571872417]
rs, fs:  2

rs, fs:  2084.673584228808   [0.0005048452503868544, 0.0031156402152076206, 0.019184643968062732, 4.535524497877503e-16, 0.44422342536130727]
rs, fs:  2080.508403629344   [0.0005027674533129294, 0.0031094787286754274, 0.01918078935113121, 4.538663389660419e-16, 0.44403714523815335]
rs, fs:  2076.351545066268   [0.0005006822705115456, 0.003103279963240275, 0.01917673090158196, 4.542015270805885e-16, 0.4438442883402336]
rs, fs:  2072.2029919121414   [0.000498589918095422, 0.0030970449442568214, 0.019172473062222885, 4.545597532830653e-16, 0.4436449633328062]
rs, fs:  2068.0627275727447   [0.0004964906091682587, 0.003090774681031122, 0.019168020184931158, 4.549428542606982e-16, 0.44343927636771774]
rs, fs:  2063.930735487016   [0.0004943845538943595, 0.003084470167179977, 0.019163376532642086, 4.553529202092842e-16, 0.4432273311329335]
rs, fs:  2059.8069991269813   [0.0004922719595731458, 0.0030781323810062083, 0.019158546281333812, 4.557919997904238e-16, 0.4430092289012503]
rs, fs:  2055

rs, fs:  1841.5586191285438   [0.0004354294488721173, 0.0028981206910223345, 0.019074273234913244, 7.772438292003943e-16, 0.42941492311209306]
rs, fs:  1837.879182553341   [0.0004346576933144044, 0.002895358512525129, 0.019070980401277506, 8.029273175689743e-16, 0.42908554121080467]
rs, fs:  1834.207097497319   [0.0004338833143349111, 0.0028925787439145222, 0.019067579879476725, 8.309808243432944e-16, 0.42875274903680693]
rs, fs:  1830.5423492721327   [0.00043310637279581987, 0.0028897817048725754, 0.019064073281148963, 8.616639777940662e-16, 0.4284165755156193]
rs, fs:  1826.884923218784   [0.00043232692938682663, 0.002886967712386796, 0.019060462189964187, 8.952636261211315e-16, 0.42807704911187294]
rs, fs:  1823.234804707564   [0.0004315450445840636, 0.0028841370806215365, 0.01905674816181421, 9.321601716340762e-16, 0.427734197830449]
rs, fs:  1819.591979137994   [0.0004307607786175646, 0.0028812901208221317, 0.019052932725049695, 9.727371100193821e-16, 0.4273880492176863]
rs, fs:  

rs, fs:  1633.3159747177551   [0.00040608105615913483, 0.002787314889767468, 0.018952012269939467, 5.404325127211223e-11, 0.4099320612631224]
rs, fs:  1630.0526072236032   [0.00040573781298605845, 0.002785847709436558, 0.018949625091689466, 9.005648394703142e-11, 0.40958276204078653]
rs, fs:  1626.7957599420536   [0.00040539292092698514, 0.002784369115660217, 0.018947172283490715, 1.5294002229850063e-10, 0.40923137728415626]
rs, fs:  1623.5454198457128   [0.0004050464286906625, 0.0027828794108355737, 0.01894465590475498, 2.647696665209482e-10, 0.4088779285816854]
rs, fs:  1620.301573933216   [0.00040469838352731743, 0.002781378867549776, 0.018942077750983548, 4.674619850230379e-10, 0.4085224469832287]
rs, fs:  1617.0642092291755   [0.00040434883016504463, 0.0027798677071686844, 0.018939439167740473, 8.421549933531039e-10, 0.4081649533351047]
rs, fs:  1613.833312784128   [0.0004039978305219538, 0.0027783462101946535, 0.018936741780486, 1.5495385409999008e-09, 0.4078054697518161]
rs, fs:

In [ ]:
plt.figure()
plt_xe_std, = plt.plot(a[3].rs, a[0], label='Standard')
plt_xe_backReact, = plt.plot(b[3].rs, b[0], label='Backreaction')
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')
plt.title(r'\bf{Ionization History}')
plt.xlabel(r'Redshift $(1+z)$')
plt.ylabel(r'Electron Fraction $x_e$ [K]')
leg = plt.legend(handles=[plt_xe_std, plt_xe_backReact],
                 title=r'$m_\chi$\bf{ = $10^{'+str(np.log10(mDM))+'}$ eV}',
                 loc=2
                )
plt.setp(leg.get_title(),fontsize=28)

plt.figure()
plt_TCMB, = plt.plot(a[3].rs, phys.TCMB(a[3].rs)/phys.kB, 'k:', label='CMB')
plt_Tm_std, = plt.plot(a[3].rs, a[1]/phys.kB, label='Standard')
plt_Tm_backReact, = plt.plot(b[3].rs, b[1]/phys.kB, label='Backreaction')

# plt.plot(b[3].rs, phys.TCMB(b[3].rs)/phys.kB, 'k:')
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')
plt.title(r'\bf{Temperature History}')
plt.xlabel(r'Redshift $(1+z)$')
plt.ylabel(r'Matter Temperature $T_m$ [K]')
leg2 = plt.legend(handles=[plt_TCMB, plt_Tm_std, plt_Tm_backReact],
                  title=r'$m_\chi$\bf{ = $10^{'+str(np.log10(mDM))+'}$ eV}',
                  loc=2
                 )
plt.setp(leg2.get_title(),fontsize=28)